# Imports

In [13]:
# Data manipulation and analysis
import pandas as pd
import numpy as np

# Data visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Machine learning
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Deep learning with TensorFlow
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.utils import to_categorical

# Deep learning with PyTorch
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torchvision import models
from torchvision.models import resnet50, ResNet50_Weights
from torchvision.models import resnet18, ResNet18_Weights

# Natural Language Processing (NLP)
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# Audio Processing
import librosa
import librosa.display
import librosa.core
from scipy.io import wavfile as wav
from scipy.fftpack import fft
from scipy import signal

# Miscellaneous
import os
import re
import time
import pickle
from io import BytesIO

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Creating Spectrograms of the Audio Files

In [18]:
directory = "/content/drive/My Drive/Machine Learning/COSMOS/FINAL_PROJECT/DER/ekman6_audio/"
folders = ["anger/", "disgust/", "fear/", "joy/", "sadness/", "surprise/"]

count = 0

def save_spectrogram(read_path, write_path):
  y, sr = librosa.load(read_path)
  y = y[:100000000] # shorten audio a bit for speed

  window_size = 1024
  window = np.hanning(window_size)
  stft  = librosa.stft(y, n_fft=window_size, hop_length=512, window=window)
  out = 2 * np.abs(stft) / np.sum(window)

  # For plotting headlessly
  from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas

  fig = plt.Figure()
  canvas = FigureCanvas(fig)
  ax = fig.add_subplot(111)
  p = librosa.display.specshow(librosa.amplitude_to_db(out, ref=np.max), ax=ax, y_axis='log', x_axis='time')
  fig.savefig(write_path)

for i in range(300):
  j = (int)(i/50)
  rp = "/content/drive/My Drive/Machine Learning/COSMOS/FINAL_PROJECT/DER/ekman6_audio/" + folders[j] + str(i) + ".mp3"
  wp = "/content/drive/My Drive/Machine Learning/COSMOS/FINAL_PROJECT/DER/ekman6_spectro/" + folders[j] + str(i) + ".jpg"
  print(rp)
  save_spectrogram(rp, wp)

/content/drive/My Drive/Machine Learning/COSMOS/FINAL_PROJECT/DER/ekman6_audio/anger/0.mp3
/content/drive/My Drive/Machine Learning/COSMOS/FINAL_PROJECT/DER/ekman6_audio/anger/1.mp3
/content/drive/My Drive/Machine Learning/COSMOS/FINAL_PROJECT/DER/ekman6_audio/anger/2.mp3
/content/drive/My Drive/Machine Learning/COSMOS/FINAL_PROJECT/DER/ekman6_audio/anger/3.mp3
/content/drive/My Drive/Machine Learning/COSMOS/FINAL_PROJECT/DER/ekman6_audio/anger/4.mp3
/content/drive/My Drive/Machine Learning/COSMOS/FINAL_PROJECT/DER/ekman6_audio/anger/5.mp3
/content/drive/My Drive/Machine Learning/COSMOS/FINAL_PROJECT/DER/ekman6_audio/anger/6.mp3
/content/drive/My Drive/Machine Learning/COSMOS/FINAL_PROJECT/DER/ekman6_audio/anger/7.mp3
/content/drive/My Drive/Machine Learning/COSMOS/FINAL_PROJECT/DER/ekman6_audio/anger/8.mp3
/content/drive/My Drive/Machine Learning/COSMOS/FINAL_PROJECT/DER/ekman6_audio/anger/9.mp3
/content/drive/My Drive/Machine Learning/COSMOS/FINAL_PROJECT/DER/ekman6_audio/anger/10.mp

# Sampling Frames from Video Files

In [19]:
import cv2

# the following was stolen from StackOverflow
def get_frames(readpath, writepath):
    cap = cv2.VideoCapture(readpath)
    i = 0
    property_id = int(cv2.CAP_PROP_FRAME_COUNT)
    length = int(cv2.VideoCapture.get(cap, property_id))
    # a variable to set how many frames you want to skip
    frame_skip = (int)(length/30) # every length/30 frames, we make another frame
    # a grand total of 30 frames for each video
    # a variable to keep track of the frame to be saved
    frame_count = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        if i > frame_skip - 1:
            frame_count += 1
            if not cv2.imwrite(writepath + "_" + str(frame_count) + ".jpg", frame):
              print(readpath + " " + writepath)
              raise Exception("Could not write image")
            i = 0
            continue
        i += 1
    cap.release()
    cv2.destroyAllWindows()

In [23]:
directory = "/content/drive/My Drive/Machine Learning/COSMOS/FINAL_PROJECT/DER/ekman6/"

id = 0

for i in range(6):
  path = directory + folders[i]
  cnt = 0
  for file in os.listdir(path):
    filename = os.fsdecode(file)
    rp = path + filename
    wp = "/content/drive/My Drive/Machine Learning/COSMOS/FINAL_PROJECT/DER/ekman6_split/" + folders[i] + str(id)
    get_frames(rp, wp)
    cnt += 1
    id += 1
    print(id)
    if cnt == 50:
      break

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [ ]:
directory = "/content/drive/My Drive/Machine Learning/COSMOS/FINAL_PROJECT/DER/ekman6_split/"



# Text Processing

# Combining into torch.datasets

In [25]:
indices = []
labels = []

for i in range(300):
  indices.append(torch.tensor([i]))
  labels.append(torch.tensor([(int)(i/50)]))

indices = torch.stack(indices)
labels = torch.stack(labels)

dset = torch.utils.data.TensorDataset(indices, labels)
torch.save(dset, "ekman.pt")